<a href="https://colab.research.google.com/github/prabsh/Fake_news_detection_paper/blob/main/Sentence_based_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -qq transformers
import pandas as pd
from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
import torch
# from transformers import BertModel
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
!pip install nltk

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_path = "/content/drive/MyDrive/FakeNewsNet-master/code/fake-news-detection-master/fakenewsnet_dataset/train.csv"
val_path ="/content/drive/MyDrive/FakeNewsNet-master/code/fake-news-detection-master/fakenewsnet_dataset/test.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(val_path)

In [5]:
train_df

,Unnamed: 0,text,class
0,13020,hoping that gwen stefani gives birth to blake ...,1
1,1813,read more less getty images it was the split h...,0
2,11145,vodafone hutchison australia and tpg have agre...,1
3,5926,one day before his 49th birthday earlier this ...,0
4,12328,did you trip on your way over? cara delevingne...,1
...,...,...,...
10917,1544,this morning drake announced the awesomely nam...,0
10918,10112,"american actress, screenwriter, and novelist c...",1
10919,8030,khloé kardashian is over the unsolicited advic...,0
10920,12292,date nights and therapy are what ellen degener...,1


In [6]:
train_df["text"][1]

'read more less getty images it was the split heard around woke twitter , when common and political activist and commentator angela rye broke up. the oscar-winning rapper, whose known for remaining good friends with his exes — even hosting an event with ex serena williams earlier this week — said he learned a lot from the nearly yearlong relationship when asked. “first of all, i learned that she’s just an amazing, incredible woman,” the hunter killer star tells essence. “it’s also a process, and a journey to get to understanding how to be full in a relationship, and be a full and complete person as much as possible,” he continued, adding that it was “fun to have a partner who you can really enjoy and can be really friends [with].”although the couple began dating in the summer of 2017 , rye confirmed that the two had indeed broken up back in march after many noticed the 39-year-old cnn correspondent didn’t walk the red carpet with common, 46, at the oscars. “we have and will always be f

In [7]:
##getting a single document from our dataset
document = train_df["text"][0]
document

'hoping that gwen stefani gives birth to blake shelton’s baby some day!? then you need to check out these new exclusive details! nothing would make us happier than seeing gwen stefani, 48, and blake shelton, 41, welcome a child together. seriously, how incredible would that be?! now we’re learning that this dream might be coming true! “gwen’s body has changed very subtly this month and those close to her suspect she could be pregnant,” a source tells hollywoodlife.com exclusively. “her friends have not seen her touch a glass of wine or any alcohol in weeks and they think gwen has been glowing lately.” omg! so exciting! the source added that her clothes have also gotten baggier in recent days as well, leading to more heated speculation. “those that see her daily are convinced she might finally be pregnant with blake’s baby,” they added. “she has always wanted a girl and it’s clear to everyone gwen really loves blake a lot and they have always talked about having their own family togethe

###TORCH TEXT


In [8]:
!pip install torchtext
!python -m spacy download en_core_web_sm 
from torchtext.legacy.data  import Field,TabularDataset,BucketIterator,LabelField
import spacy

     |████████████████████████████████| 12.0 MB 5.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [9]:
!pip install  sentence-transformers
import nltk
from sentence_transformers import SentenceTransformer
nltk.download("punkt")
model = SentenceTransformer('all-mpnet-base-v2') 
sentences  = nltk.tokenize.sent_tokenize(document)
n  = len(sentences)
embeddings = model.encode(sentences)

     |████████████████████████████████| 79 kB 3.4 MB/s 
     |████████████████████████████████| 3.5 MB 32.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.8 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
     |████████████████████████████████| 596 kB 55.2 MB/s 
     |████████████████████████████████| 6.5 MB 40.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=1970a916a69a224fbebe2870879e4a0ed2d3a5b8c64320815373304921cdc4b7
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
##converting embedding to tensor
torch_embeddings = torch.FloatTensor(embeddings)
torch_embeddings.shape

torch.Size([16, 768])

In [11]:
F_embeddings = torch_embeddings

In [12]:
##REDUCING THE DIMENSION OF F_embedding from 384 to some lower dimension m for ease of process
m = 300
model = nn.Sequential(
    nn.Linear(768,m),
    nn.ReLU(),
)

X = model(F_embeddings)
print(X.shape)

#transpose of X
X_t = torch.transpose(X,0,1)
X_t.shape

torch.Size([16, 300])


torch.Size([300, 16])

##CONSTRUCTION OF W MATRIX

In [13]:
##making a diaginal square matrix of size m which is used while calculating the dot products of Xi,Xj embedding
a = torch.randn(m)
U = torch.diag(a)
U.shape

torch.Size([300, 300])

In [14]:
## W = X*U*X^T
##Wij = weight of edge connecting embedding Xi and Xj
Y = torch.matmul(X, U)
print(Y.shape)
W = torch.matmul(Y,X_t)
W.shape

torch.Size([16, 300])


torch.Size([16, 16])

##CORE SENTENCE EXTRACTION

In [15]:
##CORE SENTENCE EXTRACTION  The sentence which has  maximum sum of edges from it 
weight_sum = torch.sum(W,dim=1)
print(weight_sum)
core_sent = torch.argmax(weight_sum)
core_sent

tensor([ 0.0214, -0.0557,  0.0111, -0.0090,  0.0420, -0.0085,  0.0002, -0.0911,
        -0.0431,  0.0652,  0.1393, -0.0644,  0.0943,  0.0546, -0.0020,  0.0623],
       grad_fn=<SumBackward1>)


tensor(10)

##RELEVANT AND IRRELEVEANT SENTENCE EXTRACTION

In [16]:
core_sent_embedding = F_embeddings[core_sent]
##FINDING COSINE SIMILARITY OF OTHER SENTENCES WITH CORE SENTENCE
cosine_sim = torch.matmul(F_embeddings,core_sent_embedding)
print(cosine_sim.shape)

torch.Size([16])


In [17]:
##THE TOP 30% SENTENCES WITH HIGHEST COSINE SIM ARE RELEVANT OTHERS IRRELVANT
cosine_sim = cosine_sim.tolist()
sentences = [i for i in range(n)]
sentences.sort(key = lambda x: cosine_sim[x],reverse =True)
print(sentences)

[10, 14, 9, 15, 2, 0, 6, 13, 5, 11, 8, 12, 4, 1, 7, 3]


In [18]:
#sentences.pop(0)
#relevant_sentences = sentences[:(n-1)//3 +1]
#irrelevant_sentences = sentences[(n-1)//3 +1:]

In [19]:
F_embeddings[9]

tensor([-2.4420e-02,  6.3653e-02, -3.1578e-02, -3.0955e-02,  2.5899e-02,
         1.8623e-06, -7.1458e-02, -7.3655e-03, -3.0134e-02,  4.0553e-02,
        -1.3466e-03,  5.4951e-02, -1.5985e-02,  1.6279e-02, -5.1260e-02,
         3.9377e-02,  1.3199e-02,  6.0610e-02, -3.1959e-02,  4.9321e-02,
        -3.6361e-03,  2.4999e-02,  4.8536e-03,  2.2598e-02, -6.8880e-02,
        -3.4662e-02, -1.5907e-02,  3.7394e-02, -3.0578e-02, -4.0868e-02,
         1.5923e-02,  1.9718e-02,  1.5108e-02,  1.4296e-02,  1.5545e-06,
         1.0825e-02, -4.8172e-03,  2.3362e-02, -4.6788e-02, -4.2378e-02,
        -5.2053e-02, -1.7879e-02,  3.0377e-02,  8.5457e-03,  4.4616e-02,
         4.9497e-02,  1.4310e-02,  2.8538e-02, -5.5597e-02, -6.3900e-02,
         1.8784e-02, -2.1096e-02, -4.3349e-03, -1.1925e-02,  5.0134e-02,
        -3.2930e-02,  1.4744e-02,  4.1707e-02, -6.9321e-03, -8.2553e-02,
         5.2566e-02,  4.7385e-02,  4.2438e-02, -9.5426e-04,  5.1505e-03,
        -1.2021e-02,  3.3333e-04,  7.8358e-03,  6.9

In [20]:
if(n>5):
    relevant_sentences = sentences[:5]
    #Input_x=torch.sum(relevant_sentences)
else:
    relevant_sentences = sentences
    #Input_x=torch.sum(relevant_sentences)
X_Input_1=X[relevant_sentences]
#X_Input_1
X_Input_2=torch.transpose(X_Input_1,0,1)
X_Input_3=torch.sum(X_Input_2,1)
X_Input_3

tensor([0.0026, 0.0000, 0.0371, 0.0406, 0.0008, 0.1384, 0.1865, 0.2509, 0.1999,
        0.0202, 0.0710, 0.0281, 0.1435, 0.0440, 0.0000, 0.1138, 0.0000, 0.0976,
        0.1520, 0.0151, 0.0826, 0.0000, 0.0000, 0.0712, 0.0000, 0.0539, 0.0307,
        0.0000, 0.0000, 0.0128, 0.0000, 0.0968, 0.0000, 0.0075, 0.0595, 0.0093,
        0.0000, 0.1132, 0.0259, 0.0077, 0.0000, 0.1177, 0.1191, 0.0373, 0.0439,
        0.0434, 0.0000, 0.0254, 0.0837, 0.0122, 0.1658, 0.0000, 0.2247, 0.2038,
        0.0203, 0.0135, 0.1568, 0.1394, 0.0483, 0.0306, 0.0000, 0.0133, 0.0334,
        0.0000, 0.1021, 0.0027, 0.0024, 0.0080, 0.0046, 0.0984, 0.0000, 0.2405,
        0.0000, 0.1589, 0.0000, 0.1831, 0.0000, 0.0182, 0.0771, 0.0105, 0.0014,
        0.0861, 0.0097, 0.0893, 0.0355, 0.0073, 0.0000, 0.0143, 0.0826, 0.0476,
        0.0000, 0.0000, 0.1817, 0.1807, 0.2276, 0.0245, 0.0049, 0.0389, 0.1332,
        0.0000, 0.0000, 0.0598, 0.0000, 0.1170, 0.0089, 0.0000, 0.0000, 0.1649,
        0.0000, 0.0225, 0.1212, 0.0301, 

# Input Process

In [21]:
def sen_processer(text,model_e):
    sentences  = nltk.tokenize.sent_tokenize(text)
    n  = len(sentences)
    embeddings = model_e.encode(sentences)
    F_embeddings = torch.FloatTensor(embeddings)
    m = 300
    model = nn.Sequential(
        nn.Linear(768,m),
        nn.ReLU(),
        )

    X = model(F_embeddings)
    #print(X.shape)

    #transpose of X
    X_t = torch.transpose(X,0,1)
    X_t.shape
    a = torch.randn(m)
    U = torch.diag(a)
    U.shape
    Y = torch.matmul(X, U)
    #print(Y.shape)
    W = torch.matmul(Y,X_t)
    W.shape
    weight_sum = torch.sum(W,dim=1)
    #print(weight_sum)
    core_sent = torch.argmax(weight_sum)
    core_sent
    core_sent_embedding = F_embeddings[core_sent]
    ##FINDING COSINE SIMILARITY OF OTHER SENTENCES WITH CORE SENTENCE
    cosine_sim = torch.matmul(F_embeddings,core_sent_embedding)
    #print(cosine_sim.shape)
    cosine_sim = cosine_sim.tolist()
    sentences = [i for i in range(n)]
    sentences.sort(key = lambda x: cosine_sim[x],reverse =True)
    if(n>5):
         relevant_sentences = sentences[:5]
        
    else:
         relevant_sentences = sentences
        
    X_Input_1=X[relevant_sentences]         
    X_Input_2=torch.transpose(X_Input_1,0,1)
    out=torch.sum(X_Input_2,1)
    return out

# Model


In [22]:
model = SentenceTransformer('all-mpnet-base-v2')


In [26]:
ip=torch.ones([10922,300])

In [ ]:
for i in range(10922):
  ip[i]=sen_processer(train_df["text"][i],model)


In [ ]:
EPOCHS = 50
BATCH_SIZE = 1
LEARNING_RATE = 0.001

In [ ]:
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(300, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [ ]:
model_bc = BinaryClassification()
print(model_bc)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model_bc.parameters(), lr=LEARNING_RATE)

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [ ]:
Y=torch.FloatTensor(train_df['class'])

In [ ]:
class TrainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

In [ ]:
train_data = TrainData(torch.FloatTensor(ip),Y)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
model_bc.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch, y_batch
        optimizer.zero_grad()
        
        y_pred = model_bc(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        loss.backward(retain_graph=True)
        optimizer.step()
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

In [ ]:
test_len=len(test_df)
X_test=torch.ones([test_len,300])
for i in range(10922):
  X_test[i]=sen_processer(test_df["text"][i],model)
Y_test=torch.FloatTensor(test_df['class'])



In [ ]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [ ]:
confusion_matrix(y_test, y_pred_list)

In [ ]:
print(classification_report(y_test, y_pred_list))